<a href="https://colab.research.google.com/github/IntroComputationalPhysics-UNT/kapitza-pendulum-firebats2000/blob/main/kapitza_pendulum.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Psuedocode

1. Define dimensionlesss equation of motion:
* $\theta''+ 2\zeta\theta' + (\epsilon^2 - \alpha cos\tau)sin\theta = 0$
* $\tau = \omega_d t$

2. Define the ODE system (RHS function):
* Input: $\tau$, state vector $y = [\theta, \theta'], parameters (\epsilon, \alpha, \zeta)$
* Output: derivatives $[\theta',\theta'']$
* function kapitza_rhs($\tau, y, \epsilon, \alpha, \zeta$):
  * $\theta = y[0]$
  * $\theta' = y[1]$
  * $\theta'' = -2\zeta\theta' - (\epsilon^2 - \alpha cos(\tau))sin(\theta)$
* return [$\theta', \theta''$]

3.  Integrate using solve_ivp:
* sol = solve_ivp(kapitza_rhs, t_span = (0, tau_end), y0 = [theta0, theta_dot0], t_eval = tau_eval, args = (eps, alpha, zeta))
* Extarct $\theta(t)$: $\theta = sol.y[0]$
* Deiscard tansient portion:
  * index_start = index corresponding to N_periods_discard
  * theta_ss = theta[index_start:]
* compute stability measure

4.  Parameter sweep over $(\epsilon, \alpha)$
* eps_vals = array of $\epsilon$ values
* alpha_vals = array of $\alpha$ values
* create 2D array S to store diagnostics: S = zeros((len(alpha_vals), len(eps_vals)))

5.  Extract approximate stability boundry $\alpha_c(\epsilon)$
* alpha_c = array of lenght(eps_cals)
* For each epsilon index j:
  * column = S[:, j]
  * find first $\alpha_i$ where diagnostic < threshhold
  * if found: alpha_c[j] = alpha_vals[i]
  * else: alpha_c[j] = NaN

6.  Fit $\alpha_c = C\epsilon^2$
* valid_indices = where(alpha_c is finite)
* C= mean(alpha_c[valid]/(eps_vals[valid]^2))

7.  Plot 2D stability map
* Use matplotlib imshow or pcolormesh:
  * x-axis = $\epsilon$
  * y-axis = $\alpha$
  * color = diagnostic S[i, j]
* overlay fitted stability curve:
  * $\alpha_{fit}(\epsilon) = C\epsilon^2$
  * plot $\epsilon$ vs $\alpha_{fit}$

8. Print summary:
* Print fitted constant c
* Print $\alpha_c(\epsilon) table
* Print comments on stable vs unstable regions